In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Jan  2 06:34:30 PST 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.5 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.866
BogoMIPS:              5851.93
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


#### Make directories

In [5]:
%%bash
mkdir /home/sam/data/virginica
mkdir /home/sam/analyses/20190103_Cvirginica_vcf_splitting

mkdir: cannot create directory ‘/home/sam/data/virginica’: File exists
mkdir: cannot create directory ‘/home/sam/analyses/20190103_Cvirginica_vcf_splitting’: File exists


#### Create variables

In [3]:
%env vcf_dir=/home/sam/data/virginica
%env wd=/home/sam/analyses/20190103_Cvirginica_vcf_splitting
%env bcftools=/home/sam/software/bcftools-1.9-62-g1ce0f47/bcftools
%env bgzip=/home/sam/software/bin/bgzip
%env tabix=/home/sam/software/bin/tabix

env: vcf_dir=/home/sam/data/virginica
env: wd=/home/sam/analyses/20190103_Cvirginica_vcf_splitting
env: bcftools=/home/sam/software/bcftools-1.9-62-g1ce0f47/bcftools
env: bgzip=/home/sam/software/bin/bgzip
env: tabix=/home/sam/software/bin/tabix


#### Check software versions

In [4]:
%%bash
${bcftools} --version
${bgzip}  --version
${tabix} --version

bcftools 1.9-62-g1ce0f47
Using htslib 1.9-44-g80f3557
Copyright (C) 2018 Genome Research Ltd.
License Expat: The MIT/Expat license
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
bgzip (htslib) 1.9-44-g80f3557
Copyright (C) 2018 Genome Research Ltd.
tabix (htslib) 1.9-44-g80f3557
Copyright (C) 2018 Genome Research Ltd.


#### Download VCF file

In [6]:
%%bash
cd ${vcf_dir}

rsync \
--archive \
--verbose \
--progress \
gannet:/volume1/seashell/eog_v2/VCF_files/Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz .

# Uncomment line below to download via web
# wget http://gannet.fish.washington.edu/seashell/eog_v2/VCF_files/Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz

ls -lh

receiving incremental file list
total 0


ERROR: module is write only
rsync error: syntax or usage error (code 1) at main.c(782) [sender=3.0.9]
rsync: connection unexpectedly closed (110 bytes received so far) [Receiver]
rsync error: error in rsync protocol data stream (code 12) at io.c(226) [Receiver=3.1.1]


In [7]:
%%bash
cd ${vcf_dir}

rsync \
--archive \
--verbose \
--progress \
gannet:/volume1/web/seashell/eog_v2/VCF_files/Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz .

# Uncomment line below to download via web
# wget http://gannet.fish.washington.edu/seashell/eog_v2/VCF_files/Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz

ls -lh

receiving incremental file list
Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz
 32,645,951,849 100%   82.68MB/s    0:06:16 (xfr#1, to-chk=0/1)

sent 30 bytes  received 32,649,937,077 bytes  86,489,899.62 bytes/sec
total size is 32,645,951,849  speedup is 1.00
total 31G
-rwxrwxrwx 1 sam users 31G Dec 19 22:03 Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz


#### Unzip VCF

In [8]:
%%bash
cd ${vcf_dir}

time \
gunzip Combined.SNP.TRSdp5g95FnDNAmaf05.vcf.gz

ls -lh

total 186G
-rwxrwxrwx 1 sam users 186G Dec 19 22:03 Combined.SNP.TRSdp5g95FnDNAmaf05.vcf



real	49m37.000s
user	20m21.120s
sys	2m30.532s


#### Sort and bgzip

In [9]:
%%bash
time \
awk '$1 ~ /^#/ {print $0;next} {print $0 | "sort -k1,1 -k2,2n"}' Combined.SNP.TRSdp5g95FnDNAmaf05.vcf |\
${bgzip} \
--threads 18 \
> ${wd}/Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz

awk: cannot open Combined.SNP.TRSdp5g95FnDNAmaf05.vcf (No such file or directory)

real	0m0.098s
user	0m0.004s
sys	0m0.000s


In [10]:
%%bash
cd ${vcf_dir}

time \
awk '$1 ~ /^#/ {print $0;next} {print $0 | "sort -k1,1 -k2,2n"}' Combined.SNP.TRSdp5g95FnDNAmaf05.vcf |\
${bgzip} \
--threads 18 \
> ${wd}/Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz

sort: write failed: /tmp/sortDMZSYb: No space left on device

real	26m25.034s
user	3m37.928s
sys	1m59.788s


No space??

In [11]:
%%bash
df -h

Filesystem                     Size  Used Avail Use% Mounted on
udev                            36G     0   36G   0% /dev
tmpfs                          7.1G  722M  6.4G  10% /run
/dev/sda6                      224G   80G  133G  38% /
tmpfs                           36G  221M   36G   1% /dev/shm
tmpfs                          5.0M  4.0K  5.0M   1% /run/lock
tmpfs                           36G     0   36G   0% /sys/fs/cgroup
/dev/sdb1                      917G  508G  363G  59% /home
cgmfs                          100K     0  100K   0% /run/cgmanager/fs
tmpfs                          7.1G  168K  7.1G   1% /run/user/1000
//owl.fish.washington.edu/web   51T   48T  2.8T  95% /mnt/owl


Deleted some stuff...

In [12]:
%%bash
df -h

Filesystem                     Size  Used Avail Use% Mounted on
udev                            36G     0   36G   0% /dev
tmpfs                          7.1G  722M  6.4G  10% /run
/dev/sda6                      224G   80G  133G  38% /
tmpfs                           36G  250M   36G   1% /dev/shm
tmpfs                          5.0M  4.0K  5.0M   1% /run/lock
tmpfs                           36G     0   36G   0% /sys/fs/cgroup
/dev/sdb1                      917G  212G  659G  25% /home
cgmfs                          100K     0  100K   0% /run/cgmanager/fs
tmpfs                          7.1G  168K  7.1G   1% /run/user/1000
//owl.fish.washington.edu/web   51T   48T  2.8T  95% /mnt/owl


In [13]:
%%bash
cd ${vcf_dir}

time \
awk '$1 ~ /^#/ {print $0;next} {print $0 | "sort -k1,1 -k2,2n"}' Combined.SNP.TRSdp5g95FnDNAmaf05.vcf |\
${bgzip} \
--threads 18 \
> ${wd}/Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz

sort: write failed: /tmp/sortrHTYQb: No space left on device

real	30m51.276s
user	3m39.152s
sys	1m58.672s


Ran out of space - again. Duh. Just realized the space limitation was in ```/tmp```. Deleting the other files I previously removed had no effect.

Need to fix this...

In [14]:
%%bash
cd ${vcf_dir}

time \
awk '$1 ~ /^#/ {print $0;next} {print $0 | "sort --temporary-directory=/home/sam/Downloads -k1,1 -k2,2n"}' Combined.SNP.TRSdp5g95FnDNAmaf05.vcf |\
${bgzip} \
--threads 18 \
> ${wd}/Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz


real	306m21.798s
user	118m7.088s
sys	30m32.388s


#### Index with ```tabix```

In [15]:
%%bash
cd ${wd}

time \
${tabix} --preset vcf Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz

ls -lh

total 33G
-rw-rw-r-- 1 sam sam  33G Jan  2 14:43 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz
-rw-rw-r-- 1 sam sam 538K Jan  2 15:14 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz.tbi



real	19m46.280s
user	18m55.812s
sys	0m22.376s


#### Fill AN/AC values

In [16]:
%%bash
cd ${wd}

export BCFTOOLS_PLUGINS=/home/sam/software/bcftools-1.9-62-g1ce0f47/plugins

time \
${bcftools} plugin fill-AN-AC Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz \
--output-type z \
--threads 18 \
--output Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz

ls -lh

total 66G
-rw-rw-r-- 1 sam sam  33G Jan  2 17:05 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz
-rw-rw-r-- 1 sam sam  33G Jan  2 14:43 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz
-rw-rw-r-- 1 sam sam 538K Jan  2 15:14 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz.tbi



real	109m50.921s
user	210m56.972s
sys	1m53.408s


In [18]:
%%bash
cd ${wd}

for file in Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz; do
  for sample in $($bcftools query -l $file); do
      bcftools view \
      --min-ac 1 \
      --output-type z \
      --samples $sample \
      --output-file ${file/.vcf*/.$sample.vcf.gz} \
      --threads 18 \
      ${file}
  done
done

bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command no

In [19]:
%%bash
cd ${wd}

for file in Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz; do
  for sample in `$bcftools query -l $file`; do
      bcftools view \
      --min-ac 1 \
      --output-type z \
      --samples $sample \
      --output-file ${file/.vcf*/.$sample.vcf.gz} \
      --threads 18 \
      ${file}
  done
done

bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command no

In [20]:
%%bash
cd ${wd}

for file in Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz; do
  for sample in $(/home/sam/software/bcftools-1.9-62-g1ce0f47/bcftools query -l $file); do
      bcftools view \
      --min-ac 1 \
      --output-type z \
      --samples $sample \
      --output-file ${file/.vcf*/.$sample.vcf.gz} \
      --threads 18 \
      ${file}
  done
done

bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command not found
bash: line 5: bcftools: command no

In [21]:
%%bash
cd ${wd}

for file in Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz; do
  for sample in $($bcftools query -l $file); do
      ${bcftools} view \
      --min-ac 1 \
      --output-type z \
      --samples $sample \
      --output-file ${file/.vcf*/.$sample.vcf.gz} \
      --threads 18 \
      ${file}
  done
done

In [23]:
%%bash

# Copy data to Gannet
cd /home/sam/analyses
time \
rsync \
--archive \
--relative \
./20190103_Cvirginica_vcf_splitting gannet:/volume1/web/Atumefaciens


real	21m56.303s
user	14m55.344s
sys	4m17.120s


In [22]:
%%bash
ls -lhtr ${wd}

total 113G
-rw-rw-r-- 1 sam sam  33G Jan  2 14:43 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz
-rw-rw-r-- 1 sam sam 538K Jan  2 15:14 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.vcf.gz.tbi
-rw-rw-r-- 1 sam sam  33G Jan  2 17:05 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.vcf.gz
-rw-rw-r-- 1 sam sam 567M Jan  2 20:28 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_1.vcf.gz
-rw-rw-r-- 1 sam sam 586M Jan  2 21:32 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_2.vcf.gz
-rw-rw-r-- 1 sam sam 588M Jan  2 22:36 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_3.vcf.gz
-rw-rw-r-- 1 sam sam 560M Jan  2 23:41 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_4.vcf.gz
-rw-rw-r-- 1 sam sam 549M Jan  3 00:45 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_5.vcf.gz
-rw-rw-r-- 1 sam sam 572M Jan  3 01:50 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CL_6.vcf.gz
-rw-rw-r-- 1 sam sam 527M Jan  3 02:54 Combined.SNP.TRSdp5g95FnDNAmaf05.sorted.ANACfill.CLP_1.vcf.gz
-rw-rw-r-- 1 sam 